# Problem Set 1

As noted in the instructions in the README (if you haven't read the README yet, please go look at it!), you can do this problem set in a stand-alone Python program or in this notebook. I can't provide any assistance installing libraries and making sure that you are using the right version of Python on your own computer so if you are not comfortable with that sort of thing, you should use Colab.

If you choose to use Colab, follow the same instructions you have been following all along: (1) rename the notebook as `Lastname_Firstname_PS1.ipynb`; (2) download and push the notebook to your GitHub repo, and (3) share your notebook with me and the TAs.

This problem set is due Friday, February 21, at 11:59pm EST. You still have to answer all the questions in the README, so please read the README. You will fail the problem set if you skip reading the README since that is where the instructions and questions are.

## 1. Getting the data
You can run the following code block to get the data. **Do not try to commit to or fork this repo or really anything else other than cloing this repo.** It's for you to get the data easily without worrying about mounting your Google Drive, which can be confusing for students.


In [1]:
! git clone https://github.com/CSCI-3349-S25/ps1.git

Cloning into 'ps1'...
remote: Enumerating objects: 2043, done.
remote: Counting objects: 100% (2043/2043), done.
remote: Compressing objects: 100% (2041/2041), done.
remote: Total 2043 (delta 13), reused 2009 (delta 2), pack-reused 0 (from 0)
Receiving objects: 100% (2043/2043), 3.57 MiB | 5.91 MiB/s, done.
Resolving deltas: 100% (13/13), done.


You can see now that you have a `ps1` directory with a few different directories in it: `pos`, `neg`, and `test`. Enter some unix code below to make sure it looks correct.

In [2]:
# unix commands to make sure you have ps1
# and that inside ps1 are pos, neg, and test
! ls
! ls ps1


ps1  sample_data
nb.py  neg  pos  README.md  test


## 2. Setting some global variables

Run the code below to set the global variables. Read the comments to understand what they are for.

In [3]:
from nltk import FreqDist
import glob
import math
import re
from textblob import TextBlob

########################
### GLOBAL VARIABLES ###
########################

## hand-crafted list of stop words
stops = {"(", ")", "--", "*", ":", "-", "may", "though", ";",
         "thing", "things", "'d", "'ll", "'m", "'ve", "'t", "'s",
         "'re", "a", "about", "above", "after", "again", "against",
         "ain", "all", "am", "an", "and", "any", "are", "aren't",
         "as", "at", "be", "because", "been", "before", "being", "below",
         "between", "both", "but", "by", "can", "could", "couldn't",
         "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down",
         "during", "each", "few", "for", "from", "further", "had", "hadn't",
         "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll",
         "he's", "her", "here", "here's", "hers", "herself", "him", "himself",
         "his", "how", "how's", "i", "i'd", "i'll", "i'm", "i've", "if", "in",
         "into", "is", "isn't", "it", "it's", "its", "itself", "just", "let's",
         "me", "mightn't", "more", "most", "mustn't", "my", "myself",
         "needn't", "no", "nor", "not", "now", "o", "of", "off", "on", "once",
         "only", "or", "other", "ought", "our", "ours", "ourselves", "out",
         "over", "own", "same", "shan't", "she", "she'd", "she'll", "she's",
         "should", "should've", "shouldn't", "so", "some", "such", "t", "than",
         "that", "that'll", "that's", "the", "their", "theirs", "them",
         "themselves", "then", "there", "there's", "these", "they", "they'd",
         "they'll", "they're", "they've", "this", "those", "through", "to",
         "too", "under", "until", "up", "very", "was", "wasn't", "we", "we'd",
         "we'll", "we're", "we've", "were", "weren't", "what", "what's",
         "when", "when's", "where", "where's", "which", "while", "who",
         "who's", "whom", "why", "why's", "will", "with", "won't", "would",
         "wouldn", "wouldn't", "y'all", "you", "you'd", "you'll", "you're",
         "you've", "your", "yours", "yourself", "yourselves", ",", ".", "!",
         "?", "'", '"', "I", "i"}

# these store the words used in positive and negative reviews
# these will be populated in read_in_training_data()
poswords = []
negwords = []

# these store the NB probability for each word in positive reviews ("positive word")
# and each word in negative reviews ("negative word")
# these will be populated in calculate_nb_probabilities()
poswordprobs = {}
negwordprobs = {}

# these store the smoothed NB probability for each word
# these will be populated in calculate_smooth_nb_probabilities()
smooth_poswordprobs = {}
smooth_negwordprobs = {}



## 3. Read in the data
The code below reads in the data. You do not need to modify this code. Read the comments to understand what it's doing.

In [4]:
#################################
### FUNCTION TO READ IN DATA ###
#################################
# You don't need to modify this.
# This returns the lists of positive words
# and negative words from the training data.

def read_in_training_data():


    ## Read in all positive reviews
    ## We create a set of unique words for each review.
    ## We then add that set of words as a list to the master list of positive words.
    positivewords = []
    allpos = glob.glob("ps1/pos/*")
    for filename in allpos:
        f = open(filename)
        thesewords = set()
        for line in f:
            words = line.rstrip().split()
            for w in words:
                if w not in stops:
                    thesewords.add(w)
        f.close()
        positivewords.extend(list(thesewords))

    print(len(positivewords), "positive tokens found!")
    print(len(set(positivewords)), "positive types found!")


    ## Read in all negative reviews
    ## We create a set of unique words for each review.
    ## We then add that set of words as a list to the master list of negative words.
    negativewords = []
    allneg = glob.glob("ps1/neg/*")
    for filename in allneg:
        f = open(filename)
        thesewords = set()
        for line in f:
            words = line.rstrip().split()
            for w in words:
                if w not in stops:
                    thesewords.add(w)
        f.close()
        negativewords.extend(list(thesewords))

    print(len(negativewords), "negative tokens found!")
    print(len(set(negativewords)), "negative types found!")
    return(positivewords, negativewords)



## 4. Writing the functions
Each code block below contains a function for predicting the sentiment of a movie review (positive or negative) in a different way. For each code block, you need to fill in code where it say `YOUR PART X CODE STARTS HERE`.

You can run each of these code cells one by one without changing anything. At the moment they don't do anything, so in the very last code block, you will get 50\% accuracy for everything.

Then you should fill in each section as indicated in the comments. You should be able to complete the problem set just be reading the instructions in the README and following the comments below.

In [5]:
## FUNCTION USING USER-DEFINED WORDS TO PREDICT SENTIMENT
# You just need to fill in your own keywords below.

def user_defined_keywords(reviewwords):


    #########################################
    ##### YOUR PART A CODE STARTS HERE ######
    #########################################

    # enter your keywords in the lists below
    positive_keywords = ["real", "beautiful", "brilliant", "glamorous", "exceptional", "great", "sweet", "nice", "craft", "engrossed"]
    negative_keywords = ["not", "bad", "worse", "*", "dumb", "grainy", "no", "sucked", "inept", "damn"]

    #########################################
    ##### YOUR PART A CODE ENDS HERE ########
    #########################################


    # If there are more positive than negative keywords,
    # return "pos". Otherwise, return "neg".

    sentiment = 0
    for w in reviewwords:
        if w in positive_keywords:
            sentiment += 1
        if w in negative_keywords:
            sentiment -=1

    if sentiment > 0:
        return "pos"

    return "neg"

In [6]:
## FUNCTION TO CALCULATE NAIVE BAYES PROBABILITIES
# You will be writing most of this function.
def calculate_nb_probabilities():

    ## GOAL: Populate these two dicts, where each
    ##      key = word from poswords or negwords (created for you above)
    ##      value = NB probability for that word in that class (calculated by you here)

    poswordprobs = {}
    negwordprobs = {}

    #########################################
    ##### YOUR PART B CODE STARTS HERE ######
    #########################################

    ## Create a FreqDist for poswords below.
    poswords_freq = FreqDist(poswords)

    ## Create a FreqDist for negwords below.
    negwords_freq = FreqDist(negwords)

    ## Loop through your poswords FreqDist, and calculate the
    ## probability of each word in the positive class, like this:
    ## P(word|pos) = count(word) / total number of positive tokens
    ## where count(word) is what you get from the FreqDist for poswords.
    ## Store the results in poswordprobs.
    ## USE LOGS!!!

    for word, count in poswords_freq.items():
        poswordprobs[word] = math.log(count / len(poswords))


    ## Now, loop through your negwords FreqDist, and calculate the
    ## probability of each word in the negative class, like this:
    ## P(word|neg) = count(word) / total number of negative tokens
    ## where count(word) is what you get from the FreqDist for negwords.
    ## Store the results in negwordprobs.
    ## USE LOGS!!!

    for word, count in negwords_freq.items():
        negwordprobs[word] = math.log(count / len(negwords))


    #########################################
    ##### YOUR PART B CODE ENDS HERE ########
    #########################################

    return (poswordprobs, negwordprobs)


In [7]:
 #find misclassifications
 # read in the test reviews
testdata = glob.glob("ps1/test/*")
for filename in testdata:
    wholereview = ""
    reviewwords = []
    with open(filename, encoding='utf8') as f:
        wholereview = f.read().rstrip()
    words = set(wholereview.split())
    for w in words:
        if w not in stops:
            reviewwords.append(w)

        # read the file name of the file to determine if its pos or neg
    filepolarity = re.sub(r"^.*?(pos|neg)-.*?$", r"\1", filename)

        # apply each classifier to that review, and check to see it's correct
    if filepolarity != user_defined_keywords(reviewwords):
      print(filename)

ps1/test/pos-cv972_26417.txt
ps1/test/neg-cv936_17473.txt
ps1/test/neg-cv935_24977.txt
ps1/test/neg-cv903_18981.txt
ps1/test/pos-cv998_14111.txt
ps1/test/neg-cv962_9813.txt
ps1/test/neg-cv907_3193.txt
ps1/test/neg-cv956_12547.txt
ps1/test/pos-cv914_28742.txt
ps1/test/pos-cv997_5046.txt
ps1/test/neg-cv941_10718.txt
ps1/test/neg-cv991_19973.txt
ps1/test/pos-cv988_18740.txt
ps1/test/neg-cv952_26375.txt
ps1/test/pos-cv917_29715.txt
ps1/test/neg-cv998_15691.txt
ps1/test/pos-cv971_10874.txt
ps1/test/pos-cv944_13521.txt
ps1/test/neg-cv977_4776.txt
ps1/test/pos-cv955_25001.txt
ps1/test/pos-cv909_9960.txt
ps1/test/neg-cv986_15092.txt
ps1/test/pos-cv942_17082.txt
ps1/test/neg-cv924_29397.txt
ps1/test/pos-cv908_16009.txt
ps1/test/neg-cv950_13478.txt
ps1/test/pos-cv925_8969.txt
ps1/test/pos-cv990_11591.txt
ps1/test/neg-cv921_13988.txt
ps1/test/neg-cv945_13012.txt
ps1/test/pos-cv982_21103.txt
ps1/test/pos-cv979_18921.txt
ps1/test/neg-cv951_11816.txt
ps1/test/pos-cv907_3541.txt
ps1/test/pos-cv918_26

In [8]:
## FUNCTION USING NAIVE BAYES PROBS TO PREDICT SENTIMENT
# You don't need to modify this method, but it relies
# on the code you  wrote above.

def naive_bayes(reviewwords):

    # default probability for unseen words
    defaultprob = math.log(0.0000000000001)

    ### POSITIVE SCORE
    posscore = poswordprobs.get(reviewwords[0], defaultprob)
    for i in range(1, len(reviewwords)):
        posscore += poswordprobs.get(reviewwords[i], defaultprob)

    ### CALCULATE NEGATIVE SCORE
    negscore = negwordprobs.get(reviewwords[0], defaultprob)
    for i in range(1, len(reviewwords)):
        negscore += negwordprobs.get(reviewwords[i], defaultprob)

    if (posscore - negscore) >  0:
        return "pos"

    return "neg"

In [9]:
## FUNCTION TO CALCULATE SMOOTHED NAIVE BAYES PROBABILITIES
# You will write most of this function.
def calculate_smooth_nb_probabilities():

    smooth_poswordprobs = {}
    smooth_negwordprobs = {}

    #########################################
    ##### YOUR PART C CODE STARTS HERE ######
    #########################################

    # Populate the above dictionaries just as you did in the unsmoothed
    # version, but use +1 smoothing so that you can handle unseen words.

    # +1 smoothing: when calculating the probabilities,
    # add 1 to every count found in the FreqDist for each class.
    # Divide the count by the number of types...
    #     *plus* the number of tokens for that class...
    #     *plus* 1 (for the count of the unseen word)

    # Don't forget to use logs.

    poswords_freq = FreqDist(poswords)
    negwords_freq = FreqDist(negwords)

    for word, count in poswords_freq.items():
        smooth_poswordprobs[word] = math.log((count + 1) / (len(poswords) + 1))

    for word, count in negwords_freq.items():
        smooth_negwordprobs[word] = math.log((count + 1) / (len(negwords) + 1))

    return (smooth_poswordprobs, smooth_negwordprobs)

In [10]:
## FUNCTION USING SMOOTHED NAIVE BAYES PROBS TO PREDICT SENTIMENT
# You will write most of this function.
def smooth_naive_bayes(reviewwords):

    # These are placeholders that allow the code to run.
    # You will calculate posscore and negscore below.
    posscore = 0
    negscore = 0

    # Adapt the code from naive_bayes() above to work here.
    # Use the smoothed probabilities you created above.

    # Do not forget to create a separate defaultprob for
    # unseen words for the two classes, as follows.

    # The defaultprob for each class should be
    # the log of:
    # 1 (the count of the unseen word) divided by...
    #    the number of types in that class...
    #    *plus* the number of tokens in that class...
    #    *plus* 1

    defaultprob = math.log(1 / (len(poswords) + 1))
    posscore = smooth_poswordprobs.get(reviewwords[0], defaultprob)
    for i in range(1, len(reviewwords)):
        posscore += smooth_poswordprobs.get(reviewwords[i], defaultprob)

    negscore = smooth_negwordprobs.get(reviewwords[0], defaultprob)
    for i in range(1, len(reviewwords)):
        negscore += smooth_negwordprobs.get(reviewwords[i], defaultprob)

    #########################################
    ##### YOUR PART C CODE ENDS HERE ########
    #########################################


    if (posscore - negscore) >  0:
        return "pos"

    return "neg"

In [11]:
testdata = glob.glob("ps1/test/*")
for filename in testdata:
    wholereview = ""
    reviewwords = []
    with open(filename, encoding='utf8') as f:
        wholereview = f.read().rstrip()
    words = set(wholereview.split())
    for w in words:
        if w not in stops:
            reviewwords.append(w)

        # read the file name of the file to determine if its pos or neg
    filepolarity = re.sub(r"^.*?(pos|neg)-.*?$", r"\1", filename)

        # apply each classifier to that review, and check to see it's correct
    if filepolarity != naive_bayes (reviewwords) and filepolarity == smooth_naive_bayes(reviewwords):
        print(filename)

In [12]:
# prompt: #Q8: Find three reviews that were still classified incorrectly, even with smoothing. Write them out here, and then comment on why you think they might have been challenging for the classifiers.

# Find three reviews that were still classified incorrectly, even with smoothing.
# The provided code already identifies misclassified reviews.  We just need to select three.


# Example output (replace with actual misclassified reviews):
# ps1/test/neg-23.txt
# ps1/test/pos-11.txt
# ps1/test/neg-42.txt


# Potential reasons for misclassification:

# 1. Sarcasm and irony:  Classifiers struggle with sarcasm, where positive words are used to express negative sentiments or vice versa.
#    A review might contain many positive words but be highly critical due to sarcasm.  The classifier would be tricked into predicting a positive sentiment.

# 2. Domain-specific language:  Movie reviews may contain slang, jargon, or abbreviations that the classifier hasn't encountered during training.
#   The model may misinterpret the meaning of these terms, leading to incorrect classifications.

# 3. Negation handling: A word like "not" can change the meaning of subsequent words, but classifiers might miss the context around a negation.
#    For instance, "not bad" means something different from just "bad".

# 4. Ambiguity of words: Some words have multiple meanings in different contexts. The model might not correctly identify the meaning of these words based on the review content.

# 5. Limited training data: If the training data does not include examples of reviews that use similar wording patterns to the misclassified reviews, the model may not perform well.


# To analyze the specific reasons for misclassification in the three example reviews above, you should inspect the reviews themselves and compare their content with the keywords used in the classifier. Look for potential cases of sarcasm, uncommon words or phrases, negations, or ambiguous words.




In [13]:
## FUNCTION FOR APPLYING TEXTBLOB's SENTIMENT TOOL TO A REVIEW
def calculate_textblob(review):

    #########################################
    ##### YOUR PART D CODE STARTS HERE ######
    #########################################

    # First, tead the documentation for the textblob sentiment analysis here:
    # https://textblob.readthedocs.io/en/dev/quickstart.html#sentiment-analysis
    # You will not be able to do this without reading the documentation.

    # Brief instructions:
    # Create a TextBlob object.
    # Populate it to with the review (as a string not as a list of words).
    # Get the first element of its sentiment variable.
    # If it's more than the threshold 0, return pos. Otherwise return neg.

    threshold = 0.09
    blob = TextBlob(" ".join(review))
    if blob.sentiment.polarity > threshold:
        return "pos"

    #########################################
    ##### YOUR PART D CODE ENDS HERE ########
    #########################################

    return "neg"

In [14]:
## FUNCTION FOR CALCULATING THE ACCURACY OF YOUR MODELS
# You do not need to modify this code.

def calculate_accuracy():
    keywordscorrect = 0
    nbcorrect = 0
    smnbcorrect = 0
    tbcorrect = 0
    affcorrect = 0

    # read in the test reviews
    testdata = glob.glob("ps1/test/*")
    for filename in testdata:
        wholereview = ""
        reviewwords = []
        with open(filename, encoding='utf8') as f:
            wholereview = f.read().rstrip()
        words = set(wholereview.split())
        for w in words:
            if w not in stops:
                reviewwords.append(w)

        # read the file name of the file to determine if its pos or neg
        filepolarity = re.sub(r"^.*?(pos|neg)-.*?$", r"\1", filename)

        # apply each classifier to that review, and check to see it's correct
        if filepolarity == user_defined_keywords(reviewwords):
            keywordscorrect += 1

        if filepolarity == naive_bayes(reviewwords):
            nbcorrect += 1

        if filepolarity == smooth_naive_bayes(reviewwords):
            smnbcorrect += 1

        if filepolarity == calculate_textblob(reviewwords):
            tbcorrect += 1

    # report the accuracy of each classifier
    print("User keyword accuracy: ", (keywordscorrect/200))
    print("Naive Bayes accuracy: ", (nbcorrect/200))
    print("Snoothed Naive Bayes accuracy: ", (smnbcorrect/200))
    print("TextBlob accuracy: ", (tbcorrect/200))

## 5. Evaluate!

Again, you do not need to modify this. This will work without changing anything, and will continue to work as long as your functions do what they are supposed to do.

In [15]:
#####################
### RUN ALL TESTS ###
#####################

# You do not need to modify this code.

# read in the training data to get all the positive and negative words
poswords, negwords = read_in_training_data()

# calculate the naive bayes probabilities
poswordprobs, negwordprobs = calculate_nb_probabilities()

# calculate smoothed naive bayes probabilities
smooth_poswordprobs, smooth_negwordprobs = calculate_smooth_nb_probabilities()

# calculate the accuracy of all approaches
calculate_accuracy()

251965 positive tokens found!
34422 positive types found!
228452 negative tokens found!
32227 negative types found!
User keyword accuracy:  0.61
Naive Bayes accuracy:  0.725
Snoothed Naive Bayes accuracy:  0.845
TextBlob accuracy:  0.74
